In [1]:
import os
import pandas as pd
from tqdm.auto import tqdm
import re
import pickle
from keras.preprocessing.text import Tokenizer
import time, datetime

def timestamp():
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S:')
    return st

print(timestamp(), 'Libraries Imported')

Using TensorFlow backend.


17:33:42: Libraries Imported


In [2]:
DOCSTRING_REGEX = re.compile(r"\"\"\"(.|\n)*?\"\"\"")
COMMENT_REGEX = re.compile(r"#.*")
NOT_WORDS_REGEX = re.compile(r"[^a-zA-Z]")
EXTRAWHITESPACES_REGEX = re.compile(r"\s+")

PYTHON_DIR = 'D:\\Users\\Ritvik\\Anaconda3\\envs\\datascience\\lib'
count = 0
with open('E:/Scrapped-Data/MyPyBot/python_code.txt','a') as input_f:
    for path, directories, files in tqdm(os.walk(PYTHON_DIR)):
        for file in files:
            if file.endswith('.py'):
                try:
                    with open(os.path.join(path, file), 'r') as data_f:
                        contents = DOCSTRING_REGEX.sub('', data_f.read())
                        contents = COMMENT_REGEX.sub('', contents)
                        contents = NOT_WORDS_REGEX.sub(' ', contents)
                    input_f.write(EXTRAWHITESPACES_REGEX.sub(' ', contents))
                    input_f.write('\n')
                    count += 1
                except Exception as e:
                    print(os.path.join(path, file) ,str(e))
                    pass
                    
print(timestamp(), 'Data Extracted', count) 

D:\Users\Ritvik\Anaconda3\envs\datascience\lib\functools.py 'charmap' codec can't decode byte 0x81 in position 301: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\shlex.py 'charmap' codec can't decode byte 0x81 in position 1434: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\sre_compile.py 'charmap' codec can't decode byte 0x90 in position 1139: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\distutils\tests\test_archive_util.py 'charmap' codec can't decode byte 0x81 in position 2764: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\distutils\tests\test_log.py 'charmap' codec can't decode byte 0x8d in position 1113: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\distutils\tests\test_msvccompiler.py 'charmap' codec can't decode byte 0x81 in position 3315: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\pydoc_data\

D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\pandas\tests\io\json\test_readlines.py 'charmap' codec can't decode byte 0x9d in position 872: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\pandas\tests\io\msgpack\test_pack.py 'charmap' codec can't decode byte 0x81 in position 813: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\pandas\tests\io\parser\python_parser_only.py 'charmap' codec can't decode byte 0x9d in position 2790: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\pandas\tests\io\parser\test_read_fwf.py 'charmap' codec can't decode byte 0x9d in position 2864: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\pandas\tests\io\parser\usecols.py 'charmap' codec can't decode byte 0x81 in position 12643: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\panda

D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\spacy\lang\fa\tokenizer_exceptions.py 'charmap' codec can't decode byte 0x81 in position 1851: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\spacy\lang\fa\lemmatizer\_adjectives.py 'charmap' codec can't decode byte 0x81 in position 81: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\spacy\lang\fa\lemmatizer\_adjectives_exc.py 'charmap' codec can't decode byte 0x81 in position 986: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\spacy\lang\fa\lemmatizer\_lemma_rules.py 'charmap' codec can't decode byte 0x9d in position 1098: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\spacy\lang\fa\lemmatizer\_nouns.py 'charmap' codec can't decode byte 0x81 in position 344: character maps to <undefined>
D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\spacy

In [3]:
with open('E:/Scrapped-Data/MyPyBot/python_code.txt','r') as f:
    data = f.read()
data = data.split()

print(timestamp(), 'File Read', len(data))

16:42:57: File Read 28079384


In [4]:
word_counts = pd.Series(data).value_counts()
print(timestamp(), 'Words Counted')
word_counts.value_counts().head()

16:43:10: Words Counted


1    3029706
2     333440
3     117725
5      60612
6      53873
dtype: int64

In [5]:
tokenizer = Tokenizer()

In [6]:
tokenizer.fit_on_texts(data)
print(timestamp(), 'Tokenizer Trained')

count_thres = 5
low_count_words = [w for w,c in tqdm(tokenizer.word_counts.items()) if c < count_thres]
for w in tqdm(low_count_words):
    del tokenizer.word_index[w]
    del tokenizer.word_docs[w]
    del tokenizer.word_counts[w]
print(timestamp(), 'Tokenizer Updated')

pickle.dump(tokenizer, open('E:/Scrapped-Data/MyPyBot/Tokenizer.pkl', 'wb'))
print(timestamp(), 'Tokenizer Pickled')

16:49:55: Tokenizer Trained



16:50:06: Tokenizer Updated
16:50:12: Tokenizer Pickled


In [10]:
sorted(tokenizer.word_counts.items(), key=lambda x: x[1], reverse=True)

[('self', 589763),
 ('n', 296203),
 ('r', 272315),
 ('s', 227595),
 ('def', 211098),
 ('if', 198260),
 ('d', 193884),
 ('none', 171843),
 ('m', 164534),
 ('k', 160940),
 ('a', 153363),
 ('x', 152431),
 ('return', 148671),
 ('name', 147620),
 ('t', 143667),
 ('i', 132064),
 ('l', 122467),
 ('in', 116559),
 ('is', 115668),
 ('import', 111348),
 ('b', 111222),
 ('g', 109445),
 ('f', 101613),
 ('assert', 96724),
 ('z', 93062),
 ('y', 90721),
 ('from', 90536),
 ('not', 83552),
 ('u', 80240),
 ('np', 79162),
 ('c', 77585),
 ('for', 76128),
 ('e', 70101),
 ('v', 67876),
 ('type', 65821),
 ('get', 65603),
 ('h', 62513),
 ('p', 62438),
 ('value', 59413),
 ('test', 58025),
 ('else', 53800),
 ('data', 53705),
 ('result', 52458),
 ('false', 50105),
 ('equal', 47471),
 ('true', 47150),
 ('class', 46772),
 ('as', 44528),
 ('to', 43368),
 ('path', 42960),
 ('index', 42481),
 ('raise', 41551),
 ('array', 41436),
 ('dtype', 40855),
 ('and', 37735),
 ('shape', 37664),
 ('ops', 36970),
 ('len', 36131),
 

In [8]:
tokenizer = pickle.load(open('E:/Scrapped-Data/MyPyBot/Tokenizer.pkl', 'rb'))

data = tokenizer.texts_to_sequences(tqdm(data))
print(timestamp(), 'Data Tokenized')

data = [item for sublist in tqdm(data) for item in sublist]
print(timestamp(), 'Token List Flattened', len(data))

print(len(tokenizer.word_counts))


16:58:55: Data Tokenized



16:59:22: Token List Flattened 23876929
405753


In [9]:
pd.DataFrame({'tokens':data}).to_csv('E:/Scrapped-Data/MyPyBot/py_tokens.csv', index=False)
print('Data Saved')

Data Saved


In [2]:
tokens = pd.read_csv('E:/Scrapped-Data/MyPyBot/py_tokens.csv')['tokens'].values
print(timestamp(), 'File Read')

train_len = 3 + 1
sequences = []
for i in tqdm(range(train_len, len(tokens)+1)):
    seq = tokens[i-train_len:i]
    sequences.append(seq)
    
print(timestamp(), 'Sequenced', len(sequences))

pd.DataFrame(sequences).to_csv('E:/Scrapped-Data/MyPyBot/sequences.csv', index=False)
print(timestamp(), 'File Saved')

17:34:18: File Read



17:35:03: Sequenced 23876926 209183992
17:58:54: File Saved
